In [1]:
import os
import time
import warnings
import numpy as np
import random as rnd
import pandas as pd
from collections import defaultdict

# Librería Genética
from deap import base, creator, tools, algorithms

from sklearn.utils import shuffle
# Subfunciones de estimadores
from sklearn.base import clone
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][30]
from sklearn.base import is_classifier
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][535]
from sklearn.model_selection._validation import _fit_and_score
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_validation.py][346]
from sklearn.model_selection._search import BaseSearchCV
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][386]
from sklearn.model_selection._search import check_cv
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py][1866]
from sklearn.model_selection._search import _check_param_grid
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][343]
from sklearn.metrics.scorer import check_scoring
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/scorer.py][250]
from sklearn.utils.validation import _num_samples
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][105]
from sklearn.utils.validation import indexable
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][208]
from multiprocessing import Pool, Manager

# Selección para estimadores
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Metricas para estimadores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

# Estimadores
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# EAS

In [66]:
def _initIndividual(individuo, maxints):
	"""[Iniciar Individuo]
	Arguments:
		pcls {[creator.Individual]} -- [Iniciar individuo con indices aleatorios]
		maxints {[params_size]} -- [lista de máximos índices]
	Returns:
		[creator.Individual] -- [Creación de individuo]
	"""
	return individuo(rnd.randint(0, maxint) for maxint in maxints)

def _mutIndividual(individual, maxints, prob_mutacion):
	"""[Mutación Individuo]
	Arguments:
		individual {[creator.Individual]} -- [Individuo de población]
		maxints {[lista]} -- [lista de máximos índices]
		prob_mutacion {[float]} -- [probabilidad de mutación del gen]
	Returns:
		[creator.Individual] -- [Individuo mutado]
	"""
	for i in range(len(maxints)):
		if rnd.random() < prob_mutacion:
			individual[i] = rnd.randint(0, maxints[i])
	return individual,

def _cxIndividual(ind1, ind2, prob_cruce):
	"""[Cruce de Individuos]
	Arguments:
		ind1 {[creator.Individual]} -- [Individuo 1]
		ind2 {[creator.Individual]} -- [Individuo 2]
		indpb {[float]} -- [probabilidad de emparejar]
		gene_type {[list]} -- [tipos de dato de los parámetros, CATEGORICO o NUMERICO]
	Returns:
		[creator.Individual,creator.Individual] -- [nuevos Individuos]
	"""
	CATEGORICO = 1  # int o str
	NUMERICO = 2  # float
	for i in range(len(ind1)):
		if rnd.random() < prob_cruce:
			sorted_ind = sorted([ind1[i], ind2[i]])
			ind1[i] = rnd.randint(sorted_ind[0], sorted_ind[1])
			ind2[i] = rnd.randint(sorted_ind[0], sorted_ind[1])
	return ind1, ind2

def _individual_to_params(individual, frecuencias):
	names_ = frecuencias[0].columns.values
	dataset = pd.DataFrame()
	dataset[names_[0]] = frecuencias[individual[0]][names_[0]]
	dataset[names_[1]] = frecuencias[individual[1]][names_[1]]
	dataset[names_[2]] = frecuencias[individual[2]][names_[2]]
	dataset[names_[3]] = frecuencias[individual[3]][names_[3]]
	dataset[names_[4]] = frecuencias[individual[4]][names_[4]]
	dataset[names_[5]] = frecuencias[0][names_[5]]
	# separación de data en X,y 
	y = dataset[names_[5]]
	del dataset[names_[5]]
	X = dataset
	return X,y


def _evalFunction(individual, frecuencias, scorer, num_folds, score_cache):
	"""[Evaluación del modelo]
	Arguments:
		individual {[creator.Individual]} -- [Individuo]
		frecuencias {[list]} -- [lista de dataframes]
		X {[array]} -- [Input]
		y {[array]} -- [Output]
		scorer {[string]} -- [Parámetro de evaluación, precisión]
		cv {[int | cross-validation]} -- [Especificación de los folds]
		uniform {[boolean]} -- [True hace que la data se distribuya uniformemente en los folds]
		fit_params {[dict | None]} -- [parámetros para estimator.fit]
	Keyword Arguments:
		verbose {integer} -- [Mensajes de descripción] (default: {0})
		error_score {numerico} -- [valor asignado si ocurre un error en fitting] (default: {'raise'})
		score_cache {dict} -- [description] (default: {{}})
	"""
	X, y = _individual_to_params(individual, frecuencias)
	score = 0
	n_test = 0
	paramkey = str(np.array(individual)+1)
	if paramkey in score_cache:
		score = score_cache[paramkey]
	else:
		kfold = KFold(n_splits=10, shuffle=False)
		#cv_results = cross_val_score(estimator, X, y, cv=kfold, scoring=scoring)
		cv_results = cross_val_score(individual.est, X, y, cv=kfold, scoring=scorer)
		score = np.mean(cv_results)
		score_cache[paramkey] = score
	return (score,)

In [67]:
class EvolutiveSearchCV:
	def __init__(self, estimator, scoring=None, num_folds=4,
				refit=True, verbose=False, population_size=50,
				gene_mutation_prob=0.2, gene_crossover_prob=0.5,
				tournament_size=3, generations_number=10, gene_type=None,
				n_jobs=1, uniform=True, error_score='raise',
				fit_params={}):
		# Parámetros iniciales
		self.estimator = estimator
		#self.params = params
		self.scoring = scoring
		self.num_folds = num_folds
		self.refit = refit
		self.verbose = verbose
		self.population_size = population_size
		self.gene_mutation_prob = gene_mutation_prob
		self.gene_crossover_prob = gene_crossover_prob
		self.tournament_size = tournament_size
		self.generations_number = generations_number
		self.gene_type = gene_type
		self.n_jobs = n_jobs
		self.uniform = uniform
		self.error_score = error_score
		self.fit_params = fit_params
		# Parámetros adicionales
		self._individual_evals = {}
		self.all_history_ = None
		self.all_logbooks_ = None
		self._cv_results = None
		self.best_score_ = None
		self.best_params_ = None
		self.scorer_ = None
		#self.score_cache = {}
		self.__manager = Manager()
		self.score_cache = self.__manager.dict()
		# Fitness [base.Fitness], objetivo 1
		creator.create("FitnessMax", base.Fitness, weights=(1.0,))
		# Individuo [list], parámetros:est, FinessMax
		creator.create("Individual", list, est=clone(self.estimator), fitness=creator.FitnessMax)
	#@property
	def cv_results_(self):
		if self._cv_results is None:
			out = defaultdict(list)
			gen = self.all_history_
			# Get individuals and indexes, their list of scores,
			# and additionally the name_values for this set of parameters
			idxs, individuals, each_scores = zip(*[(idx, indiv, np.mean(indiv.fitness.values))
											for idx, indiv in list(gen.genealogy_history.items())
											if indiv.fitness.valid and not np.all(np.isnan(indiv.fitness.values))])
			#name_values, _, _ = _get_param_types_maxint(self.params)
			# Add to output
			#out['param_index'] += [p] * len(idxs)
			out['index'] += idxs
			#out['params'] += [_individual_to_params(indiv, name_values) for indiv in individuals]
			out['params'] += [str(np.add(indiv,1)) for indiv in individuals]
			out['mean_test_score'] += [np.nanmean(scores) for scores in each_scores]
			out['std_test_score'] += [np.nanstd(scores) for scores in each_scores]
			out['min_test_score'] += [np.nanmin(scores) for scores in each_scores]
			out['max_test_score'] += [np.nanmax(scores) for scores in each_scores]
			out['nan_test_score?'] += [np.any(np.isnan(scores)) for scores in each_scores]
			self._cv_results = out
		return self._cv_results
	@property
	def best_index_(self):
		return np.argmax(self.cv_results_['max_test_score'])
	# fit y refit general
	def fit(self, frecuencias):
		self.best_estimator_ = None
		self.best_mem_score_ = float("-inf")
		self.best_mem_params_ = None
		#_check_param_grid(self.params)
		self._fit(frecuencias)
		#if self.refit:
		#	self.best_estimator_ = clone(self.estimator)
		#	#self.best_estimator_.set_params(**self.best_mem_params_)
		#	self.best_estimator_.fit(frecuencias)
	# fit individual
	def _fit(self, frecuencias):
		self._cv_results = None  # Indicador de necesidad de actualización
		self.scorer_ = check_scoring(self.estimator, scoring=self.scoring)
		#n_samples = _num_samples(X)
		# verificar longitudes x,y 
		#if _num_samples(y) != n_samples:
		#	raise ValueError('Target [y], data [X] dont agree')
		#cv = check_cv(self.cv, y=y, classifier=is_classifier(self.estimator))
		toolbox = base.Toolbox()
		# name_values = lista de parametros, gene_type = [1:categorico; 2:numérico], maxints = size(parametros)
		#name_values, self.gene_type, maxints = _get_param_types_maxint(parameter_dict)
		maxints = [5]*5
		#if self.verbose:
		#	print("Tipos: %s, rangos: %s" % (self.gene_type, maxints))
		# registro de función Individuo
		toolbox.register("individual", _initIndividual, creator.Individual, maxints=maxints)
		# registro de función Población
		toolbox.register("population", tools.initRepeat, list, toolbox.individual)
		# Paralelísmo, create pool
		if not isinstance(self.n_jobs, int):
			self.n_jobs=1
		pool = Pool(self.n_jobs)
		toolbox.register("map", pool.map)
		# registro de función Evaluación
		toolbox.register("evaluate", _evalFunction,
						frecuencias=frecuencias,
						scorer=self.scorer_, num_folds=4, 
						score_cache=self.score_cache)
		# registro de función Cruce
		toolbox.register("mate", _cxIndividual, prob_cruce=self.gene_crossover_prob)
		# registro de función Mutación
		toolbox.register("mutate", _mutIndividual, prob_mutacion=self.gene_mutation_prob, maxints=maxints)
		# registro de función Selección
		toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)
		# Creación de Población
		pop = toolbox.population(n=self.population_size)
		# Mejor Individuo que ha existido
		hof = tools.HallOfFame(1)
		# Stats
		stats = tools.Statistics(lambda ind: ind.fitness.values)
		stats.register("avg", np.nanmean)
		stats.register("min", np.nanmin)
		stats.register("max", np.nanmax)
		stats.register("std", np.nanstd)
		# Genealogía
		hist = tools.History()
		# Decoración de operadores de variaznza
		toolbox.decorate("mate", hist.decorator)
		toolbox.decorate("mutate", hist.decorator)
		hist.update(pop)
		# Posibles combinaciones
		if self.verbose:
			print('--- Evolve in {0} possible combinations ---'.format(np.prod(np.array(maxints) + 1)))
		pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=self.gene_crossover_prob, 
										mutpb=self.gene_mutation_prob,
										ngen=self.generations_number, 
										stats=stats,
										halloffame=hof, 
										verbose=self.verbose)
		#pop, logbook = algorithms.eaGenerateUpdate(toolbox,
		#								ngen=self.generations_number, stats=stats,
		#								halloffame=hof, verbose=self.verbose)
		# Save History
		self.all_history_ = hist
		self.all_logbooks_ = logbook
		# Mejor score y parametros
		current_best_score_ = hof[0].fitness.values[0]
		current_best_params_ = str(hof[0]) #_individual_to_params(hof[0], name_values)
		#if self.verbose:
		#	print("Best individual is: %s\nwith fitness: %s" % (
		#		current_best_params_, current_best_score_))
		if current_best_score_ > self.best_mem_score_:
			self.best_mem_score_ = current_best_score_
			self.best_mem_params_ = current_best_params_
		# fin paralelización, close pool
		pool.close()
		pool.join()
		self.best_score_ = current_best_score_
		self.best_params_ = current_best_params_

In [68]:
"""
maxints = [5]*5
scoring = "accuracy"
num_folds = 4
maxints = [5]*5
model = KNeighborsClassifier()
def __init__(self, estimador, scoring=None, num_folds=4,
				refit=True, verbose=False, population_size=50,
				gene_mutation_prob=0.2, gene_crossover_prob=0.5,
				tournament_size=3, generations_number=10, gene_type=None,
				n_jobs=1, uniform=True, error_score='raise',
				fit_params={}):
"""
seed = 7
frecuencias = []
frecuencias.append(pd.read_csv('Tx_0x01'))
frecuencias.append(pd.read_csv('Tx_0x02'))
frecuencias.append(pd.read_csv('Tx_0x03'))
frecuencias.append(pd.read_csv('Tx_0x04'))
frecuencias.append(pd.read_csv('Tx_0x05'))
frecuencias.append(pd.read_csv('Tx_0x06'))
frecuencias.append(pd.read_csv('Tx_0x07'))
frecuencias[0] = shuffle(frecuencias[0], random_state=seed).reset_index(drop=True)
frecuencias[1] = shuffle(frecuencias[1], random_state=seed).reset_index(drop=True)
frecuencias[2] = shuffle(frecuencias[2], random_state=seed).reset_index(drop=True)
frecuencias[3] = shuffle(frecuencias[3], random_state=seed).reset_index(drop=True)
frecuencias[4] = shuffle(frecuencias[4], random_state=seed).reset_index(drop=True)
frecuencias[5] = shuffle(frecuencias[5], random_state=seed).reset_index(drop=True)
frecuencias[6] = shuffle(frecuencias[6], random_state=seed).reset_index(drop=True)
num_jobs=8
#n_neighbors = 5 7 11
#weights = 'distance'
#algorithm = 'kd_tree' 'ball_tree'
gs = EvolutiveSearchCV(estimator=KNeighborsClassifier(n_neighbors=11, weights = 'distance', algorithm = 'kd_tree'), scoring="accuracy", num_folds=10, n_jobs=num_jobs,
                        verbose=True, refit=True, 
                        population_size=100, 
                        gene_mutation_prob=0.3, 
                        gene_crossover_prob=0.5,
                        tournament_size=4,
                        generations_number=12)

In [69]:
gs.fit(frecuencias)

--- Evolve in 7776 possible combinations ---
gen	nevals	avg     	min    	max     	std      
0  	100   	0.763227	0.65848	0.838799	0.0403739
1  	64    	0.796568	0.674576	0.86255 	0.0297848
2  	57    	0.812009	0.744211	0.86255 	0.0251431
3  	61    	0.820793	0.680619	0.867182	0.0295187
4  	68    	0.826507	0.735968	0.86657 	0.0271453
5  	74    	0.834793	0.715639	0.86657 	0.0259517
6  	55    	0.846039	0.746631	0.869995	0.0262022
7  	59    	0.848164	0.712216	0.866576	0.0288245
8  	78    	0.848022	0.746834	0.866773	0.0264607
9  	63    	0.851753	0.762325	0.867182	0.022693 
10 	71    	0.852063	0.735761	0.867182	0.0283103
11 	71    	0.854797	0.763728	0.869995	0.0222641
12 	73    	0.85576 	0.765341	0.869995	0.0211914


## Resultados Algoritmos Genéticos

In [70]:
dfEAS = pd.DataFrame.from_dict(dict(gs.score_cache), orient='index')
dfEAS.reset_index(level=0, inplace=True)
display(dfEAS.sort_values([0],ascending=False).head(30))
# Guarda el dataframe en un archivo csv
dfEAS.sort_values([0],ascending=False).to_csv('eas.csv', sep=',', index=False) 

,index,0
321,[6 1 2 3 5],0.869995
210,[6 1 3 6 5],0.867182
353,[6 1 2 6 5],0.866773
348,[6 1 3 3 5],0.866576
260,[6 1 2 3 6],0.866570
373,[6 1 2 3 1],0.864964
299,[6 1 2 6 6],0.863557
107,[6 1 6 6 6],0.862550
421,[6 1 2 3 4],0.862144
419,[6 1 2 1 6],0.859126


# EDAS

In [40]:

# The problem to optimize
def getAccuracy( frecuencias, individual, estimator, score_cache ):
	X,y = _createDataset(frecuencias, individual)
	score = 0
	scorer = "accuracy"
	paramkey = str(np.int32(individual)+1)
	if paramkey in score_cache:
		score = score_cache[paramkey]
	else:
		kfold = KFold(n_splits=10, shuffle=False)
		cv_results = cross_val_score(estimator, X, y, cv=kfold, scoring=scorer)
		score = np.mean(cv_results)
		score_cache[paramkey] = score
	return score

def _createDataset(frecuencias, values):
    names_ = frecuencias[0].columns.values
    dataset = pd.DataFrame()
    dataset[names_[0]] = frecuencias[int(values[0])][names_[0]]
    dataset[names_[1]] = frecuencias[int(values[1])][names_[1]]
    dataset[names_[2]] = frecuencias[int(values[2])][names_[2]]
    dataset[names_[3]] = frecuencias[int(values[3])][names_[3]]
    dataset[names_[4]] = frecuencias[int(values[4])][names_[4]]
    dataset[names_[5]] = frecuencias[0][names_[5]]
    # separación de data en X,y 
    y = dataset[names_[5]]
    del dataset[names_[5]]
    X = dataset
    return X,y

class eda:
	def __init__(self, of, frecuencias, estimator):
		# Algorithm parameters
		self.iterations = 20
		self.sample_size = 100
		self.select_ratio = 0.5
		self.epsilon = 10e-6

		# class members
		self.objective_function = of
		self.dimensions = 5
		self.sample = []
		self.means = []
		self.stdevs = []	

		self.debug = False
		# aditional parameters
		self.frecuencias = frecuencias
		self.estimator = estimator
		self.score_cache = {}


	def sample_sort(self): 
		# sort rows on the last column
		self.sample = self.sample[ np.argsort( self.sample[:,-1], 0 ) ]


	def dispersion_reduction(self):
		self.sample_sort()

		# number of points to select
		nb = int( np.floor( self.sample_size * self.select_ratio ) )

		# selection
		#self.sample = self.sample[:nb]
		self.sample = self.sample[self.sample_size-nb:]

		if self.debug:
		    print ("dispersion reduction")
		    print (str(self.sample))
		    print


	def estimate_parameters( self ):
		# points sub array (without values)
		mat = self.sample[:,:self.dimensions]
		
		# row means (axis 0 in scipy)
		self.means = np.mean( mat, 0 )
		
		# row standard deviation
		self.stdevs = np.std( mat, 0 )

		if self.debug:
		    print ("estimate parameters")
		    print ("\tmean=" +str(self.means))
		    print ("\tstd-dev=" + str(self.stdevs))
		    print


	def draw_sample(self):
		# for each variable to optimize
		for i in range(self.dimensions):
			# if the dispersion is null
			if self.stdevs[i] == 0.0:
				# set it to a minimal value
				self.stdevs[i] = self.epsilon
		
		# empty sample
		self.sample = np.zeros( (self.sample_size, self.dimensions+1) )
		
		# for each point
		for i in range( self.sample_size ):
			# draw in random normal
			p = np.random.normal( self.means, self.stdevs )
			p = np.array([0 if i<0 else (5 if i>5 else i) for i in p])
			# put it into the sample
			self.sample[i][:self.dimensions] = np.round(p)%(self.dimensions+1)

		if self.debug:
		    print ("draw sample")
		    print (self.sample)
		    print


	def evaluate(self):
		# for each point
		for i in range( self.sample_size ):
			d = self.dimensions
			# call the objective function
			#   the third element is the result of the objective function call
			#   taking the first two elements as variables
			r = self.objective_function( self.frecuencias, self.sample[i][:d], self.estimator, self.score_cache )
			self.sample[i][-1] = r

		if self.debug:
		    print ("evaluate")
		    print (self.sample)
		    print


	def run(self):
		# uniform initialization
		self.sample = np.random.rand( self.sample_size, self.dimensions+1 )
		# cosmetic
		#self.sample = self.sample * 200 - 100
		top_freq = 6
		self.sample = np.floor(np.random.rand(self.sample_size, self.dimensions +1)*top_freq)
		
		if self.debug:
		    print ("initialization")
		    print (self.sample)
		    print

		self.evaluate()

		
		i = 0
		while i < self.iterations:
			if self.debug:
			    print ("iteration",i)
			    print

			i += 1
			self.dispersion_reduction()
			print("iter["+str(i)+"]-top1: "+str(self.sample[-1]))
			self.estimate_parameters()
			self.draw_sample()
			self.evaluate()
            # print top 1
			self.sample_sort()


		# sort the final sample
		self.sample_sort()
		# output the optimum
		ranking = self.sample_size
		#print ("#[ Configuración ]\t Accuracy")
		#for i in range(ranking):
		#	linea = str(self.sample[-i-1][:-1]+1) + "\t" +str(self.sample[-i-1][-1])
		#	print(linea)


In [41]:
#n_neighbors = 5 7 11
#weights = 'distance'
#algorithm = 'kd_tree' 'ball_tree'
estimator = KNeighborsClassifier(n_jobs=8, weights = 'distance', n_neighbors = 5, algorithm = 'kd_tree')
a = eda( getAccuracy, frecuencias, estimator )
a.run()

iter[1]-top1: [ 5.          5.          2.          2.          3.          0.85207941]
iter[2]-top1: [ 5.          0.          1.          5.          4.          0.86617163]
iter[3]-top1: [ 5.          0.          2.          0.          2.          0.85067664]
iter[4]-top1: [ 5.          0.          2.          2.          2.          0.85711446]
iter[5]-top1: [ 5.          4.          1.          2.          1.          0.85207941]
iter[6]-top1: [ 5.          3.          1.          2.          1.          0.85550643]
iter[7]-top1: [ 5.          0.          1.          2.          1.          0.86274664]
iter[8]-top1: [ 5.          0.          1.          2.          3.          0.86134144]
iter[9]-top1: [ 5.          0.          1.          2.          1.          0.86274664]
iter[10]-top1: [ 5.          0.          2.          2.          2.          0.85711446]
iter[11]-top1: [ 5.          0.          1.          2.          4.          0.86999619]
iter[12]-top1: [ 5.          0

## Resultados EDAS

In [42]:
dfEDAS = pd.DataFrame.from_dict(a.score_cache, orient='index')
dfEDAS.reset_index(level=0, inplace=True)
display(dfEDAS.sort_values([0],ascending=False).head(30))
# Guarda el dataframe en un archivo csv
dfEDAS.sort_values([0],ascending=False).to_csv('edas.csv', sep=',', index=False) 

,index,0
657,[6 1 2 3 5],0.869996
141,[6 1 2 6 5],0.866172
655,[6 1 3 3 5],0.865569
672,[6 1 2 3 6],0.864757
564,[6 1 2 3 2],0.862747
601,[6 1 2 3 4],0.861341
659,[6 1 2 3 1],0.859729
302,[6 1 3 3 3],0.857114
517,[6 4 2 3 2],0.855506
646,[6 4 2 3 6],0.854900


# Combinatorial

In [ ]:
import itertools
combinaciones = [p for p in itertools.product(range(6), repeat=5)]
score_GridSearch = {}
for individual in combinaciones:
    getAccuracy( frecuencias, individual, estimator, score_GridSearch )

## Resulados Combinación Combinatorial

In [ ]:
dfGComb = pd.DataFrame.from_dict(score_GridSearch, orient='index')
dfGComb.reset_index(level=0, inplace=True)
display(dfGComb.sort_values([0],ascending=False).head(30))
# Guarda el dataframe en un archivo csv
dfGComb.sort_values([0],ascending=False).to_csv('combi.csv', sep=',', index=False)